<center><h1>Exploring the Competitors</h1></center>

### Part 1. Get location data using Foursquare

[Foursquare Places API](https://location.foursquare.com/products/places-api/) is very usefule online application used by many developers & other application like Uber etc. In this project you can used it to retrieve informtion about the places present in the neighborhoods of Toronto. The API returns a JSON file and we need to turn that into a data-frame. Here I’ve chosen similar businesses(pet grooming) for each neighborhood within a radius of 2.5km.

You will need to create an account with Foursquare to access the API.  It is free to sign up, and you get $200 free credit.



In [1]:
import requests
import pandas as pd

In [2]:
toronto_DF = pd.read_csv('postalcode.csv')

In [3]:
display(toronto_DF)

,Borough,Postalcode,Neighbourhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,North Toronto West,43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160
...,...,...,...,...,...
98,York,M6C,Humewood-Cedarvale,43.693781,-79.428191
99,York,M6E,Caledonia-Fairbanks,43.689026,-79.453512
100,York,M6M,"Del Ray, Mount Dennis, Keelsdale and Silverthorn",43.691116,-79.476013
101,York,M6N,"Runnymede, The Junction North",43.673185,-79.487262


In [4]:
# Set up your API key and other constants for Foursquare API
API_KEY = 'fsq3qoHPok1qOXZWHaA96hYcXSSajCW9rIT0MOW9RBIKrWQ='  # Replace with your actual Foursquare API key
LIMIT = 20  # Maximum number of venues to return
radius = 2000  # Search radius in meters

# a function to loop through each neighbourhood in the postalcode.csv file, and search for places
def getNearbyVenues(names, postalcode, latitudes, longitudes, radius):
    venues_list = []

    for name, postalcode, lat, lng in zip(names, postalcode, latitudes, longitudes):
        print(f"Processing neighborhood: {name}")

        # Skip if latitude or longitude is missing
        if pd.isnull(lat) or pd.isnull(lng):
            print(f"Skipping {name} due to missing latitude or longitude.")
            continue

        # Create the API request URL and parameters
        # Change the params here to customize your search, https://docs.foursquare.com/developer/reference/place-search
        # The following example looks for pet groomers in the neighbourhood
        # I am searching with categories ID 18027-Pilates Studio
        url = 'https://api.foursquare.com/v3/places/search'
        params = {
            'll': f'{lat},{lng}',
            'radius': radius,
            'limit': LIMIT,
            'categories': 18027
        }

        # Set up headers with the API key
        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY
        }

        # Make the GET request
        response = requests.get(url, headers=headers, params=params)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {name}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for venue in results.get('results', []):
            # Extract venue information
            venue_name = venue.get('name')
            venue_id = venue.get('fsq_id')
            venue_location = venue.get('geocodes', {}).get('main', {})
            venue_lat = venue_location.get('latitude')
            venue_lng = venue_location.get('longitude')
            venue_categories = venue.get('categories', [])
            venue_category = venue_categories[0]['name'] if venue_categories else 'Unknown'

            # Append to the list
            venues_list.append([
                name,
                postalcode,
                lat,
                lng,
                venue_name,
                venue_id,
                venue_lat,
                venue_lng,
                venue_category
            ])

    # Create a DataFrame from the list
    nearby_venues = pd.DataFrame(venues_list, columns=[
        'Neighbourhood',
        'Postalcode',
        'Neighborhood Latitude',
        'Neighborhood Longitude',
        'Venue',
        'fsq_id',
        'Venue Latitude',
        'Venue Longitude',
        'Venue Category'
    ])

    return nearby_venues

# Example usage with your DataFrame 'toronto_DF'
# Ensure 'toronto_DF' has the columns 'Neighborhood', 'Latitude', 'Longitude'
toronto_venues = getNearbyVenues(
    names=toronto_DF['Neighbourhood'],
    postalcode=toronto_DF['Postalcode'],
    latitudes=toronto_DF['Latitude'],
    longitudes=toronto_DF['Longitude'],
    radius=radius
)

# Display the first few rows of the resulting DataFrame
toronto_venues.head(20)


Processing neighborhood: Lawrence Park
Processing neighborhood: Davisville North
Processing neighborhood: North Toronto West
Processing neighborhood: Davisville
Processing neighborhood: Moore Park, Summerhill East
Processing neighborhood: Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Processing neighborhood: Roselawn
Processing neighborhood: Forest Hill North & West
Processing neighborhood: The Annex, North Midtown, Yorkville
Processing neighborhood: Rosedale
Processing neighborhood: St. James Town, Cabbagetown
Processing neighborhood: Church and Wellesley
Processing neighborhood: Regent Park, Harbourfront
Processing neighborhood: Garden District, Ryerson
Processing neighborhood: St. James Town
Processing neighborhood: Berczy Park
Processing neighborhood: Central Bay Street
Processing neighborhood: Richmond, Adelaide, King
Processing neighborhood: Harbourfront East, Union Station, Toronto Islands
Processing neighborhood: Toronto Dominion Centre, Design Exchange
Proc

,Neighbourhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,M4N,43.728020,-79.388790,Lili Viola Pilates Inc,31075a42d66e4d8e534d83a9,43.731336,-79.403600,Pilates Studio
1,Lawrence Park,M4N,43.728020,-79.388790,Essence Pilates,4e7518bc7d8b917602eaf409,43.714116,-79.400148,Pilates Studio
2,Lawrence Park,M4N,43.728020,-79.388790,Lawrence Park Chiropractic & Physiotherapy Clinic,c8f9c54b09c341cf648d4929,43.718590,-79.401190,Massage Clinic
3,Davisville North,M4P,43.712751,-79.390197,Essence Pilates,4e7518bc7d8b917602eaf409,43.714116,-79.400148,Pilates Studio
4,Davisville North,M4P,43.712751,-79.390197,Pilates-Stott Pilates Studio Equipment Sales &...,4b9842fdf964a520063835e3,43.706095,-79.398446,Pilates Studio
5,Davisville North,M4P,43.712751,-79.390197,Striation 6,4e278ca9a809ec0663e5d6e5,43.698445,-79.394745,Pilates Studio
6,Davisville North,M4P,43.712751,-79.390197,Stott Pilates,6d28bac5ad31477861e67c99,43.706625,-79.398109,Pilates Studio
7,Davisville North,M4P,43.712751,-79.390197,Lawrence Park Chiropractic & Physiotherapy Clinic,c8f9c54b09c341cf648d4929,43.718590,-79.401190,Massage Clinic
8,Davisville North,M4P,43.712751,-79.390197,Body Harmonics Pilates,e2e88849bbb44eb6dc10c9ba,43.706397,-79.401256,Pilates Studio
9,Davisville North,M4P,43.712751,-79.390197,North Movement Studio,191a326590db451f73ff78d6,43.707479,-79.375497,Pilates Studio


In [5]:
toronto_venues = toronto_venues[toronto_venues['Venue Category'] == 'Pilates Studio']
toronto_venues.head(20)

,Neighbourhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,M4N,43.728020,-79.388790,Lili Viola Pilates Inc,31075a42d66e4d8e534d83a9,43.731336,-79.403600,Pilates Studio
1,Lawrence Park,M4N,43.728020,-79.388790,Essence Pilates,4e7518bc7d8b917602eaf409,43.714116,-79.400148,Pilates Studio
3,Davisville North,M4P,43.712751,-79.390197,Essence Pilates,4e7518bc7d8b917602eaf409,43.714116,-79.400148,Pilates Studio
4,Davisville North,M4P,43.712751,-79.390197,Pilates-Stott Pilates Studio Equipment Sales &...,4b9842fdf964a520063835e3,43.706095,-79.398446,Pilates Studio
5,Davisville North,M4P,43.712751,-79.390197,Striation 6,4e278ca9a809ec0663e5d6e5,43.698445,-79.394745,Pilates Studio
6,Davisville North,M4P,43.712751,-79.390197,Stott Pilates,6d28bac5ad31477861e67c99,43.706625,-79.398109,Pilates Studio
8,Davisville North,M4P,43.712751,-79.390197,Body Harmonics Pilates,e2e88849bbb44eb6dc10c9ba,43.706397,-79.401256,Pilates Studio
9,Davisville North,M4P,43.712751,-79.390197,North Movement Studio,191a326590db451f73ff78d6,43.707479,-79.375497,Pilates Studio
10,North Toronto West,M4R,43.715383,-79.405678,Essence Pilates,4e7518bc7d8b917602eaf409,43.714116,-79.400148,Pilates Studio
11,North Toronto West,M4R,43.715383,-79.405678,Pilates-Stott Pilates Studio Equipment Sales &...,4b9842fdf964a520063835e3,43.706095,-79.398446,Pilates Studio


In [6]:
# The total number of pilates studios in Toronto
toronto_venues['Venue'].nunique()

50

In [7]:
# Oberserving the number of competitors in each neighbourhood
group_counts= toronto_venues.groupby('Neighbourhood')['Venue']
group_counts.count()

Neighbourhood
Bayview Village                                                                                                                            1
Bedford Park, Lawrence Manor East                                                                                                          2
Berczy Park                                                                                                                                6
Brockton, Parkdale Village, Exhibition Place                                                                                               2
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport                                 4
Central Bay Street                                                                                                                        15
Christie                                                                                                                                   5

In [8]:
group_counts=group_counts.count()

max_group = group_counts.idxmax()
max_group

'Ontario Provincial Government'

### Part 2. Interactive leaflet map using coordinate data.


In [9]:
import folium # map rendering library

In [10]:
# My example below shows pet groomers in Garden District, Ryerson
target = max_group

search_area = toronto_venues[toronto_venues['Neighbourhood'] == target]
latitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Latitude']
longitude = toronto_DF[toronto_DF['Neighbourhood'] == target]['Longitude']
display(search_area)

,Neighbourhood,Postalcode,Neighborhood Latitude,Neighborhood Longitude,Venue,fsq_id,Venue Latitude,Venue Longitude,Venue Category
271,Ontario Provincial Government,M7A,43.662301,-79.389494,Imprint Pilates,4bb13f68f964a520d7863ce3,43.646432,-79.395056,Pilates Studio
272,Ontario Provincial Government,M7A,43.662301,-79.389494,Sagrario Pilates Studio,4aeaed64f964a520b9bc21e3,43.665463,-79.381116,Pilates Studio
273,Ontario Provincial Government,M7A,43.662301,-79.389494,Yorkville Pilates,552fcdcb498e1648d9296c98,43.669954,-79.395815,Pilates Studio
274,Ontario Provincial Government,M7A,43.662301,-79.389494,Pilates For Life,4b27ba32f964a520068924e3,43.671943,-79.390847,Pilates Studio
275,Ontario Provincial Government,M7A,43.662301,-79.389494,Coyne Pilates,4bd8a06003979c745323bdd2,43.672889,-79.384777,Pilates Studio
276,Ontario Provincial Government,M7A,43.662301,-79.389494,Eden Pilates,4f147559d5fb04999e94a665,43.674763,-79.388350,Pilates Studio
277,Ontario Provincial Government,M7A,43.662301,-79.389494,Body Mason,55fb37b3498edd6515930db3,43.674836,-79.388127,Pilates Studio
278,Ontario Provincial Government,M7A,43.662301,-79.389494,Shas Yoga & Pilates Studio,4fe399f7e4b07215b0198bdb,43.675472,-79.403337,Pilates Studio
279,Ontario Provincial Government,M7A,43.662301,-79.389494,Live Body Wellness Consulting,4cc5c1383ed5199c91524b1e,43.646378,-79.395796,Pilates Studio
280,Ontario Provincial Government,M7A,43.662301,-79.389494,Walker Dance Studios,aadffd01ad104bb8624857b6,43.666475,-79.384971,Pilates Studio


In [11]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, venue, neighborhood in zip(search_area['Venue Latitude'], search_area['Venue Longitude'], search_area['Venue'], search_area['Neighbourhood']):
    label = '{},{}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

/opt/conda/lib/python3.11/site-packages/folium/utilities.py:94: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(coord)
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:100: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(float(coord)):
/opt/conda/lib/python3.11/site-packages/folium/utilities.py:102: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return [float(x) for x in coords]


In [17]:
# Set search_area to the entire toronto_venues dataset
search_area = toronto_venues

# Set the map center to the average coordinates of the dataset
latitude = toronto_venues['Venue Latitude'].mean()
longitude = toronto_venues['Venue Longitude'].mean()

# Create the map
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add markers for all venues in the dataset
for lat, lng, venue, neighborhood in zip(search_area['Venue Latitude'], search_area['Venue Longitude'], search_area['Venue'], search_area['Neighbourhood']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

# Display the resulting map
map_toronto


### Part 3. Customer Reviews
It seems like there is one one pet groomer withing 2km of Scarborough Village, I am interested in seeing customer's comments on Funny Bunny.  I can use [Foursquare's Place Tips API.](https://docs.foursquare.com/developer/reference/place-tips)


In [12]:
# a function to loop through the list of pilates studios in the neighborhood and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"

        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY
        }

        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')


            # Append to the list
            venues_tips.append([
                venue,
                tip_id,
                tip_created,
                tip_text
            ])


    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])


    return nearby_tips



nearby_tips = getTips(
    venues=search_area['Venue'],
    fsq_ids=search_area['fsq_id']
)

'''
for venue, fsq_id in zip(venues, fsq_ids):
    nearby_tips=getTips()
'''
# Display the first few rows of the resulting DataFrame
nearby_tips.head()

Processing venue: Imprint Pilates
Processing venue: Sagrario Pilates Studio
Processing venue: Yorkville Pilates
Processing venue: Pilates For Life
Processing venue: Coyne Pilates
Processing venue: Eden Pilates
Processing venue: Body Mason
Processing venue: Shas Yoga & Pilates Studio
Processing venue: Live Body Wellness Consulting
Processing venue: Walker Dance Studios
Failed to get data for Walker Dance Studios. Status code: 404
Error message: invalid place specified: aadffd01ad104bb8624857b6
Processing venue: Mandy Mintz
Failed to get data for Mandy Mintz. Status code: 404
Error message: invalid place specified: ff7e4f2da4b740f1e88d516f
Processing venue: Pilates With Stav
Failed to get data for Pilates With Stav. Status code: 404
Error message: invalid place specified: 5e3d496e14ef4c636a892e2d
Processing venue: Ugly Duckling Pilates
Failed to get data for Ugly Duckling Pilates. Status code: 404
Error message: invalid place specified: 20e05b15a08046913a6c96df
Processing venue: Ugly Duc

,venue_name,id,created_at,text
0,Imprint Pilates,52c96ef8498e300bd8f28b28,2014-01-05T14:40:56.000Z,The New Year is a great time to refocus your g...
1,Imprint Pilates,5217d7f3498e58df1cee00bb,2013-08-23T21:45:23.000Z,4 week Beginners to Yoga | Thursdays September...
2,Imprint Pilates,5217d960498e47d55cdfc99b,2013-08-23T21:51:28.000Z,"One On One Interview with Anita Ivic, Owner of..."
3,Imprint Pilates,52c96fc4498e2b12ea8d5b6f,2014-01-05T14:44:20.000Z,5 Week Advanced Reformer Programs - $130 You'r...
4,Imprint Pilates,5217da3611d22f4dab2a8e13,2013-08-23T21:55:02.000Z,Pilates focuses primarily on posture and spine...


In [13]:
nearby_tips.to_csv('nearby_tips_pilates.csv')

In [14]:
# a function to loop through the list of pilates studios and compile all the comments related to them
def getTips(venues, fsq_ids):
    venues_tips = []

    for venue, fsq_id in zip(venues, fsq_ids):
        print(f"Processing venue: {venue}")


        # Create the API request URL and parameters
        url = f"https://api.foursquare.com/v3/places/{fsq_id}/tips"

        headers = {
            "Accept": "application/json",
            "Authorization": API_KEY
        }

        # Make the GET request
        response = requests.get(url, headers=headers)

        # Check if the request was successful
        if response.status_code != 200:
            print(f"Failed to get data for {venue}. Status code: {response.status_code}")
            print(f"Error message: {response.text}")
            continue

        results = response.json()

        # Parse the JSON response
        for tip in results:
            # Extract venue information
            tip_id = tip.get('id')
            tip_created = tip.get('created_at')
            tip_text = tip.get('text')


            # Append to the list
            venues_tips.append([
                venue,
                tip_id,
                tip_created,
                tip_text
            ])


    # Create a DataFrame from the list
    nearby_tips = pd.DataFrame(venues_tips, columns=[
        'venue_name',
        'id',
        'created_at',
        'text'
    ])

    return nearby_tips

all_tips = getTips(
    venues=toronto_venues['Venue'],
    fsq_ids=toronto_venues['fsq_id']
)


Processing venue: Lili Viola Pilates Inc
Processing venue: Essence Pilates
Processing venue: Essence Pilates
Processing venue: Pilates-Stott Pilates Studio Equipment Sales &Certification
Processing venue: Striation 6
Processing venue: Stott Pilates
Failed to get data for Stott Pilates. Status code: 404
Error message: invalid place specified: 6d28bac5ad31477861e67c99
Processing venue: Body Harmonics Pilates
Failed to get data for Body Harmonics Pilates. Status code: 404
Error message: invalid place specified: e2e88849bbb44eb6dc10c9ba
Processing venue: North Movement Studio
Failed to get data for North Movement Studio. Status code: 404
Error message: invalid place specified: 191a326590db451f73ff78d6
Processing venue: Essence Pilates
Processing venue: Pilates-Stott Pilates Studio Equipment Sales &Certification
Processing venue: Body Harmonics Pilates
Processing venue: Lili Viola Pilates Inc
Processing venue: Body Harmonics Pilates
Failed to get data for Body Harmonics Pilates. Status code

In [15]:
# Display the first few rows of the resulting DataFramen
new_all_tips = all_tips.drop_duplicates(subset=['id'])

new_all_tips.head(20)

,venue_name,id,created_at,text
0,Pilates-Stott Pilates Studio Equipment Sales &...,586d527932b0721b934da800,2017-01-04T19:52:25.000Z,People there are so nice! I came for my first ...
1,Pilates-Stott Pilates Studio Equipment Sales &...,4eb5f590469073bbc666b065,2011-11-06T02:48:48.000Z,"reformer classes are amazing! check out Judi, ..."
6,Body Mason,5d62a28a8fc1df000698548a,2019-08-25T15:00:26.000Z,"Great instructors, welcoming space!"
7,Coyne Pilates,4cf29e1fb3662c0fdc127a30,2010-11-28T18:23:27.000Z,"Excellent, outstanding personalized Pilates in..."
9,Pilates For Life,4d08db4352a4a090537224f2,2010-12-15T15:14:11.000Z,"If you love taking Amy's classes, but can't ma..."
16,Body Harmonics Pilates,4fa55feae4b02e6a5554e845,2012-05-05T17:14:18.000Z,PILATES is good for you and your body!!
22,Sagrario Pilates Studio,4aeaedbe70c603bb6ebe8eb4,2009-10-30T13:44:30.000Z,Call & sign up for an introductory Pilates class!
23,Sagrario Pilates Studio,4aeaedfa70c603bb6fbe8eb4,2009-10-30T13:45:30.000Z,stretch class
27,Riverdale Pilates,4be345b970c603bbfb7d9ab4,2010-05-06T22:42:01.000Z,These people know their stuff. Get ready for a...
35,Imprint Pilates,52c96ef8498e300bd8f28b28,2014-01-05T14:40:56.000Z,The New Year is a great time to refocus your g...


In [16]:
new_all_tips.to_csv('all_tips_pilates.csv')